In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Описание проекта

Ретейлер "Лента" хочет научиться оптимизировать производство товаров. Необходимо построить модели, которые будут прогнозировать спрос на товары, чтобы не было недостатка и избытка в магазинах.

# План проекта

1. Изучить данные, проверить на наличие аномалий, неправильных типов и т.д. Отчистить данные от них (при необходимости);
2. Визуализировать полученные данные и сделать вывод;
3. Обучить модели с подбором параметров и выбрать среди них наилучшую;
4. Протестировать модель с лучшим показателем метрики и сделать выводы по итогам работы;

Необходимые библиотеки:

In [ ]:
!pip install  catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import tensorflow as tf


from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

warnings.filterwarnings('ignore')
state = 17

---

# EDA

In [ ]:
# данные по магазинам
st_df = pd.read_csv(r'/content/drive/MyDrive/sp_sales_task/st_df.csv')

# данные по иерархии товаров
pr_df = pd.read_csv(r'/content/drive/MyDrive/sp_sales_task/pr_df.csv')

# данные по продажам
sales_df_train = pd.read_csv(r'/content/drive/MyDrive/sp_sales_task/sales_df_train.csv')

# календарь
calendar = pd.read_csv(r'/content/drive/MyDrive/sp_sales_task/holidays_covid_calendar.csv')

# мерджим флаг активности
sales_df_train = pd.merge(sales_df_train, st_df[['st_id','st_is_active']], on='st_id', how='left')

# выбираем только рабочие магазины
sales_df_train = sales_df_train[sales_df_train['st_is_active'] == 1]

# избавляемся от возвратных позиций
sales_df_train = sales_df_train[sales_df_train['pr_sales_in_rub'] >= 0]

# создал переменную, куда скопировал основной дф
sales_df_train_1 = sales_df_train

# удаляем все строки, где объем продаж = 0
sales_df_train_1 = sales_df_train_1[sales_df_train_1['pr_sales_in_units'] != 0]

# удаляю строки, где сумма продаж по 1 позиции меньше 15 тыс. руб.
sales_df_train_1 = sales_df_train_1[sales_df_train_1['pr_sales_in_rub'] < 15000]

# агрегирую продажи по магазинам
sales_df_train_1['price'] = sales_df_train_1['pr_sales_in_rub'] / sales_df_train_1['pr_sales_in_units']

# агрегирую продажи по магазинам
gr_shop = sales_df_train_1.groupby('st_id').agg({'pr_sales_in_rub':'sum', 'pr_sales_in_units':'sum'}).reset_index()

# информация по суммарным продажам по магазинам
gr_shop.sort_values(by='pr_sales_in_units')

In [ ]:
# находим стоимость за единицу продукции
sales_df_train_1['price'] = sales_df_train_1['pr_sales_in_rub'] / sales_df_train_1['pr_sales_in_units']

# вывожу строки, где цена = 0
sales_df_train_1[sales_df_train_1['price'] == 0].head()

# удаляю строки, где price = 0
sales_df_train_1.drop(sales_df_train_1[sales_df_train_1['price'] == 0].index, inplace=True)

# небольшие махинации для дальнейшего метода merge
sales_df_train_1.drop(columns='st_is_active', inplace=True)
calendar['calday'] = calendar['calday'].astype(str)
sales_df_train_1['date'] = pd.to_datetime(sales_df_train_1['date']).dt.strftime('%Y%m%d')

sales_df_train_1 = pd.merge(sales_df_train_1, calendar, left_on='date',
                          right_on='calday', how='left')

# приводим в приличный вид
sales_df_train_1.drop(columns='date_y', inplace=True)
sales_df_train_1.rename(columns={'date_x':'date'}, inplace=True)

# добавляем фич с номером месяца
sales_df_train_1['month'] = sales_df_train_1['calday'].str[4:6]

# добавляем фичи с товарной иерархией
sales_df_train_1 = pd.merge(sales_df_train_1, pr_df, on='pr_sku_id', how='left')

# добавляем фичи с данными по магазинам
sales_df_train_1 = pd.merge(sales_df_train_1, st_df, on='st_id', how='left')

# описательная статистика продаж без промо
sales_df_train_1[sales_df_train_1['pr_sales_type_id'] == 0]['pr_sales_in_rub'].describe()

In [ ]:
# описательная статистика продаж с промо
sales_df_train_1[sales_df_train_1['pr_sales_type_id'] == 1]['pr_sales_in_rub'].describe()

In [ ]:
print('Обычные продажи в %:',
      round(sales_df_train_1['pr_sales_type_id'].value_counts()[0]/len(sales_df_train_1) * 100))
print('Промо продажи в %:',
      round(sales_df_train_1['pr_sales_type_id'].value_counts()[1]/len(sales_df_train_1) * 100))

In [ ]:
# гистограммы
sns.distplot(sales_df_train_1[sales_df_train_1['pr_sales_type_id'] == 1]['pr_sales_in_rub'],
             label='С промо', color='red', bins=30)

sns.distplot(sales_df_train_1[sales_df_train_1['pr_sales_type_id'] == 0]['pr_sales_in_rub'],
             label='Без промо', color='black', bins=30)

plt.xlabel('Продажи в рублях')
plt.title('Гистограммы продаж с/без промо')
plt.xlim(0, 10000)
plt.legend()
plt.show()

In [ ]:
# диаграммы размаха
fig, ax = plt.subplots()

ax.boxplot(sales_df_train_1[sales_df_train_1['pr_sales_type_id'] == 0]['pr_sales_in_rub'],
           positions=[1], labels=['Без промо'])

ax.boxplot(sales_df_train_1[sales_df_train_1['pr_sales_type_id'] == 1]['pr_sales_in_rub'],
           positions=[2], labels=['С промо'])

ax.set_xlabel('Типы продаж')
ax.set_title('Диаграммы размаха продаж с/без промо')
plt.show()

In [ ]:
# продажи по месяцам
month_sales = sales_df_train_1.groupby(['year', 'month']).agg({'pr_sales_in_units':'sum'}).reset_index()

In [ ]:
plt.figure(figsize=(8, 4))
plt.bar(month_sales['month'], month_sales['pr_sales_in_units'], color='black')
plt.title("Сумма продаж за 22-23г. (в шт.)")
plt.xlabel("Месяц")
plt.ylabel("Кол-во товара")

In [ ]:
# добавляем столбик с значением выходного дня (1 - день выходной, 0 - рабочий)
sales_df_train_1['weekend'] = 0
sales_df_train_1.loc[sales_df_train_1['weekday'].isin([6, 7]), 'weekend'] = 1

In [ ]:
sales_df_train_1.groupby('weekend').agg({'pr_sales_in_units':'sum'}).reset_index()



В процессе EDA было выполнено:

- Отобраны только активные магазины по флагу;
- Были удалены возвратные позиции, строки с объемом продаж = 0, строки с продажами более 15 тыс. руб.;
- Были удалены магазины (3 шт.), где продажи были слишком малы по сравнению с другими магазинами;
- Удалены также строки, где цена = 0;
- Добавлены фичи: номер месяца, товарная иерархия, данные по магазинам;
- Из описательной статистики по продажам можно сделать выводы:
  1. Средняя покупка без промо - 532 руб., с промо 707 руб.;
  2. Лишь 40% продаж приходится на промо%.
- Построены гистрограммы продаж из которых видно, что большая часть продажь - до 2 тыс. руб.;
- На диаграммах размаха видно, что есть много аномалий, однако не все аномалии - плохо, принято решение оставить эти выбросы;
- Из графика "Сумма продаж за..." можно сделать вывод, что в Декабре продажи резко подскакивают, также рост продаж отмечается и в весенний период, когда много праздников и теплеет, летом замечен спад. Вероятно, люди уезжают кто куда и спрос на товары падает;
- Около 65% продаж приходится на рабочий день.



# Обучение моделей

In [ ]:
sales_df_train_1.drop(columns=['pr_sales_type_id', 'pr_promo_sales_in_units', 'pr_sales_in_rub', 'pr_promo_sales_in_rub', 'price', 'st_is_active'], inplace=True)

Linear Regression

In [ ]:
%%time
categorical_features = sales_df_train_1.select_dtypes(include='object')

features = sales_df_train_1.drop(columns=["pr_sales_in_units"])
target = sales_df_train_1["pr_sales_in_units"]

label_encoders = {}
for feature in categorical_features:
    label_encoder = LabelEncoder()
    features[feature] = label_encoder.fit_transform(features[feature])
    label_encoders[feature] = label_encoder


scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Модель линейной регрессии
model = LinearRegression()

# Количество временных разбиений (k)
n_splits = 5

# Создание объекта TimeSeriesSplit для временной кросс-валидации
tscv = TimeSeriesSplit(n_splits=n_splits)

# Инициализация переменной для хранения WAPE на каждой итерации
wapes = []

# Цикл для временной кросс-валидации
for train_index, test_index in tscv.split(features_scaled):
    # Разделение масштабированных данных на обучающий и тестовый наборы
    features_train, features_test = features_scaled[train_index], features_scaled[test_index]
    target_train, target_test = target.iloc[train_index], target.iloc[test_index]

    # Обучение модели на обучающем наборе
    model.fit(features_train, target_train)

    # Прогноз на тестовом наборе
    target_pred = model.predict(features_test)

    # Расчет WAPE
    wape = np.sum(np.abs(target_test - target_pred)) / np.sum(np.abs(target_test))
    wapes.append(wape)

# Вывод среднего WAPE на всех итерациях кросс-валидации
print("Средний WAPE на кросс-валидации:", np.mean(wapes))

LSTM Model

In [ ]:
%%time

# Разделение данных на обучающий и тестовый наборы (по времени)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, shuffle=False)

# Выбор категориальных признаков
cat_cols = sales_df_train_1.select_dtypes(include='object')

# Преобразование категориальных признаков с использованием OrdinalEncoder
ord_enc = OrdinalEncoder().fit(features_train[cat_cols.columns])
features_train[cat_cols.columns] = ord_enc.transform(features_train[cat_cols.columns])
features_test[cat_cols.columns] = ord_enc.fit_transform(features_test[cat_cols.columns])

# Масштабирование признаков с использованием StandardScaler
scaler = StandardScaler().fit(features_train)
features_train = scaler.transform(features_train)
features_test = scaler.transform(features_test)

# Параметры для создания временных последовательностей
n_input = 1
n_features = features_train.shape[1]

# Создание генератора временных последовательностей с использованием TimeseriesGenerator
generator = TimeseriesGenerator(features_train, target_train, length=n_input, batch_size=1)

# Параметры модели LSTM
n_steps = 14
n_features = 1

# Изменение формы данных для модели LSTM
feat_train = features_train.reshape(features_train.shape[0], features_train.shape[1], n_features)
target_train = np.array(target_train)

# Создание модели LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(200, activation='relu', input_shape=(features_train.shape[1], n_features), return_sequences=True))
lstm_model.add(LSTM(32, activation='relu'))
lstm_model.add(Dense(1))

# Компиляция модели
lstm_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Обучение модели
lstm_model.fit(features_train, target_train, epochs=20, verbose=0)

# Прогнозирование на тестовом наборе данных
pred = lstm_model.predict(features_test)

# Создание DataFrame для хранения результатов
res = target_test.to_frame()

# Добавление прогнозных значений в DataFrame
res['Pred'] = pred

# Функция для вычисления WAPE (Weighted Absolute Percentage Error)
def wape(y_true: np.array, y_pred: np.array):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# Вычисление WAPE
wape_value = wape(res['pr_sales_in_units'], res['Pred'])